
---
## Projectr : 스마트서울 도시데이터 센서(S-DoT) 유동인구 측정 정보분석 ver.01
### Description : 
    - 스마트서울 도시데이터 센서(S-DoT) 유동인구 측정 정보
    [source link](https://data.seoul.go.kr/dataList/OA-15964/S/1/datasetView.do)
    - 센서수량 및 설치장소 : 서울 전역 100대(전통시장, 주요거리, 공원, 공공시설)
    - 측정정보 : S-DoT에서 측정된 유동인구(10분단위 데이터)
    - 유동인구 센서는 WiFi AP셀 반경내 휴대폰의 MAC주소를 카운팅(50대)하거나 CCTV를 활용한 피플카운팅(50대) 방식으로 정확한 유동인구를 측정한 것이 아닙니다.
    - 데이터의 양이 많아 2023년 10월 16일부터 Sheet와 OpenAPI는 최근 한달만 출력됩니다.
### Author : Forrest Dpark (분석 담당)
### Date : 2024.05.31 ~
### Detail :  
### Update: 
    > ※ 2024.06.03 pdg : 데이터 체크 
        - api 연결
---

In [25]:
import pandas as pd ,numpy as  np # Data manipulatioin
import seaborn as sns,matplotlib.pyplot as plt # visiulization
import warnings;warnings.filterwarnings('ignore') # 경고 무시 
# 기본 세팅
def plotSetting(pltStyle):
    import matplotlib.pyplot as plt # visiulization
    import platform
    from matplotlib import font_manager, rc # rc : 폰트 변경 모듈font_manager : 폰트 관리 모듈
    plt.style.use(pltStyle)
    plt.rcParams['axes.unicode_minus'] = False# unicode 설정
    if platform.system() == 'Darwin': rc('font', family='AppleGothic') # os가 macos
    elif platform.system() == 'Windows': # os가 windows
        path = 'c:/Windows/Fonts/malgun.ttf' 
        font_name = font_manager.FontProperties(fname=path).get_name()
        rc('font', family=font_name)
    else:
        print("Unknown System")
    print("___## OS platform 한글 세팅완료 ## ___")
# graph style seaborn
plotSetting("seaborn-v0_8")


___## OS platform 한글 세팅완료 ## ___


In [26]:
def JsonDataFetch(basicURL,key,data_type,start,end):
    import urllib.request as request
    import json
    # 기상청 Rss 사이트 주소 
    query  = f"/{key}/{data_type}/IotVdata018/{start}/{end}/"
    openAPI_Address= basicURL + query
    
    # key ="646b77636970756c37314f63727871"
    # openAPI_Address = f"http://openapi.seoul.go.kr:8088/{key}/json/IotVdata018/1/2/"
    # openAPI_Address = f"http://openapi.seoul.go.kr:8088/{api_key}/xml/SPOP_ORGN_CT_IN_SE_DONG/1/5/"
    # URL 생성 
    print(openAPI_Address)
    # 다운로드 
    data = request.urlopen(openAPI_Address).read()
    print(data) ## utf 8 에 xml .. 옛날 방식 

    text = data.decode("UTF-8")
    items = json.loads(text)


    print(items)
    return items
items = JsonDataFetch(
            basicURL="http://openapi.seoul.go.kr:8088",
            key="646b77636970756c37314f63727871",
            data_type ='json',
            start = 1,
            end = 100
            )



http://openapi.seoul.go.kr:8088/646b77636970756c37314f63727871/json/IotVdata018/1/100/
b'{"IotVdata018":{"list_total_count":351903,"RESULT":{"CODE":"INFO-000","MESSAGE":"\xec\xa0\x95\xec\x83\x81 \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4"},"row":[{"MODEL_NM":"SDOT001","SERIAL_NO":"00000003005","SENSING_TIME":"2024-06-02_23:44:00","REGION":"traditional_markets","AUTONOMOUS_DISTRICT":"Gangnam-gu","ADMINISTRATIVE_DISTRICT":"Yeoksam2-dong","VISITOR_COUNT":"220","REG_DTTM":"2024-06-02 23:58:05"},{"MODEL_NM":"SDOT001","SERIAL_NO":"00000003039","SENSING_TIME":"2024-06-02_23:44:00","REGION":"traditional_markets","AUTONOMOUS_DISTRICT":"Dongjak-gu","ADMINISTRATIVE_DISTRICT":"Sadang2-dong","VISITOR_COUNT":"181","REG_DTTM":"2024-06-02 23:58:05"},{"MODEL_NM":"SDOT001","SERIAL_NO":"00000003036","SENSING_TIME":"2024-06-02_23:44:00","REGION":"traditional_markets","AUTONOMOUS_DISTRICT":"Dongjak-gu","ADMINISTRATIVE_DISTRICT":"Sang-do4-dong","VISITOR_COUNT":"279"

> api 의 data 가 json 일때 까도까도 dict 인 경우 실제로 필요한 데이터를 찾기 위해 tree 구조를 제작하여 
데이터를 확인하도록 함수를 만든다. 


In [27]:

def jsonTreeCheck(items):
    def ifDict_ReturnKeyList(data,treeNum):
        # print((type(data) != type([])))
        # print((type(data) != type({})))
        
        indentSize =2 
        indentation = " "*indentSize
        print(indentation*treeNum, f"*{treeNum}th")
        if (type(data) != type([])) and (type(data) != type({})):
            print(indentation*(treeNum+1),data)
        if type(data)==type({}):
            # print(" "*treeNum,"Dict 데이터 입니다. ")
            if len(list(data.keys()))==1:
                print(indentation*(treeNum+1),f"|-> {list(data.keys())}")
            else :
                for key in data.keys():
                    print(indentation*(treeNum+1),f"|-> ['{key}'] ({type(data[key])})")
                    ifDict_ReturnKeyList(data[key],3)
                    
            # print(" "*treeNum,"|- key list type 으로 return")
            return list(data.keys())

        elif type(data) == type([]):
            print("Array 데이터 입니다. ")
            print(f"|- >arraysize =  ")
            for i in data:
                print("|- ",i)
    ## 1st tree
    item_key_list=ifDict_ReturnKeyList(items,1)
    
    ## 2nd tree
    for key in item_key_list:
        # print(items[key])   
        ifDict_ReturnKeyList(items[key],2)
jsonTreeCheck(items)

   *1th
     |-> ['IotVdata018']
     *2th
       |-> ['list_total_count'] (<class 'int'>)
       *3th
         351903
       |-> ['RESULT'] (<class 'dict'>)
       *3th
         |-> ['CODE'] (<class 'str'>)
       *3th
         INFO-000
         |-> ['MESSAGE'] (<class 'str'>)
       *3th
         정상 처리되었습니다
       |-> ['row'] (<class 'list'>)
       *3th
Array 데이터 입니다. 
|- >arraysize =  
|-  {'MODEL_NM': 'SDOT001', 'SERIAL_NO': '00000003005', 'SENSING_TIME': '2024-06-02_23:44:00', 'REGION': 'traditional_markets', 'AUTONOMOUS_DISTRICT': 'Gangnam-gu', 'ADMINISTRATIVE_DISTRICT': 'Yeoksam2-dong', 'VISITOR_COUNT': '220', 'REG_DTTM': '2024-06-02 23:58:05'}
|-  {'MODEL_NM': 'SDOT001', 'SERIAL_NO': '00000003039', 'SENSING_TIME': '2024-06-02_23:44:00', 'REGION': 'traditional_markets', 'AUTONOMOUS_DISTRICT': 'Dongjak-gu', 'ADMINISTRATIVE_DISTRICT': 'Sadang2-dong', 'VISITOR_COUNT': '181', 'REG_DTTM': '2024-06-02 23:58:05'}
|-  {'MODEL_NM': 'SDOT001', 'SERIAL_NO': '00000003036', 'SENSING_TIME': 

> 실제 데이터는 items['IotVdata018']['row'] 에 있음

In [32]:
items

{'IotVdata018': {'list_total_count': 351903,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'MODEL_NM': 'SDOT001',
    'SERIAL_NO': '00000003005',
    'SENSING_TIME': '2024-06-02_23:44:00',
    'REGION': 'traditional_markets',
    'AUTONOMOUS_DISTRICT': 'Gangnam-gu',
    'ADMINISTRATIVE_DISTRICT': 'Yeoksam2-dong',
    'VISITOR_COUNT': '220',
    'REG_DTTM': '2024-06-02 23:58:05'},
   {'MODEL_NM': 'SDOT001',
    'SERIAL_NO': '00000003039',
    'SENSING_TIME': '2024-06-02_23:44:00',
    'REGION': 'traditional_markets',
    'AUTONOMOUS_DISTRICT': 'Dongjak-gu',
    'ADMINISTRATIVE_DISTRICT': 'Sadang2-dong',
    'VISITOR_COUNT': '181',
    'REG_DTTM': '2024-06-02 23:58:05'},
   {'MODEL_NM': 'SDOT001',
    'SERIAL_NO': '00000003036',
    'SENSING_TIME': '2024-06-02_23:44:00',
    'REGION': 'traditional_markets',
    'AUTONOMOUS_DISTRICT': 'Dongjak-gu',
    'ADMINISTRATIVE_DISTRICT': 'Sang-do4-dong',
    'VISITOR_COUNT': '279',
    'REG_DTTM': '2024-06-02 23:58:05'},
  

In [33]:
def targetDataFetching(itmes,*arg):
    targetSet = items[arg[0]][arg[1]] # targetSet 은 배열이다 
    # print(targetSet)
    targetData1 =[]
    targetData2 = []
    for content in targetSet: # tragetSet 의 원소는 dict 이다. 
        serial =content['SERIAL_NO']
        region =content['REGION']
        sensingTime =content['SENSING_TIME']
        district =content['AUTONOMOUS_DISTRICT']
        admin_distric =content['ADMINISTRATIVE_DISTRICT']
        visit_count =content['VISITOR_COUNT']
        # print("-"*40)
        # print(serial[-4:])
        print(district,admin_distric)
        targetData1.append(district)
        targetData2.append(admin_distric)
        # print(region)
        # print(sensingTime)
        # print(admin_distric)
        # print(visit_count)
    return targetData1,targetData2
distric, admin_distric = targetDataFetching( items,'IotVdata018','row')

Gangnam-gu Yeoksam2-dong
Dongjak-gu Sadang2-dong
Dongjak-gu Sang-do4-dong
Dongjak-gu Sadang2-dong
Dongjak-gu Sang-do1-dong
Gangnam-gu Yeoksam1-dong
Gangnam-gu Sinsa-dong
Seocho-gu Seocho3-dong
Dongjak-gu Sang-do1-dong
Seocho-gu Banpo3-dong
Gangnam-gu Nonhyeon1-dong
Seocho-gu Seocho4-dong
Seodaemun-gu Cheonyeon-dong
Gangnam-gu Gaepo2-dong
Seocho-gu Yangjae1-dong
Gangnam-gu Daechi4-dong
Seongdong-gu Seongsu1ga1-dong
Guro-gu Gu-ro4-dong
Guro-gu Gu-ro4-dong
Gwangjin-gu Jayang1-dong
Guro-gu Gu-ro4-dong
Seodaemun-gu Hongje3-dong
Yangcheon-gu Sinwol1-dong
Mapo-gu Seogyo-dong
Songpa-gu Jamsilbon-dong
Songpa-gu Jamsil6-dong
Yangcheon-gu Sinwol1-dong
Mapo-gu Mangwon-dong1
Seodaemun-gu Hongje3-dong
Songpa-gu Jamsilbon-dong
Yangcheon-gu Mok3-dong
Seongdong-gu Seongsu1ga2-dong
Seoul_Grand_Park meeting_bridge2
Yangcheon-gu Sinwol1-dong
Mapo-gu Mangwon-dong1
Seoul_Grand_Park valet_parking1
Nowon-gu Gongneung1-dong
Dobong-gu Chang2-dong
Dobong-gu Chang2-dong
Gangdong-gu Cheonho1-dong
Gwangjin-gu Jungg

'MODEL_NM':모델번호,    
'SERIAL_NO':시리얼',    
'SENSING_TIME': 측정시간    
'REGION': 지역  
'AUTONOMOUS_DISTRICT':자치구,   
'ADMINISTRATIVE_DISTRICT'행정동:     
'VISITOR_COUNT':방문자수,   
'REG_DTTM': 등록일  
> 측정시간 , 지역, 자치구, 행정동, 방문자수 를 컬럼으로 뽑는다


In [34]:
import csv

# 데이터 리스트
data = [
    "강남구 역삼2동",
    "동작구 사당2동",
    "동작구 상도4동",
    "동작구 사당2동",
    "동작구 상도1동",
    "강남구 역삼1동",
    "강남구 신사동",
    "서초구 서초3동",
    "동작구 상도1동",
    "서초구 반포3동",
    "강남구 논현1동",
    "서초구 서초4동",
    "서대문구 천연동",
    "강남구 개포2동",
    "서초구 양재1동",
    "강남구 대치4동",
    "성동구 성수1가1동",
    "구로구 구로4동",
    "구로구 구로4동",
    "광진구 자양1동",
    "구로구 구로4동",
    "서대문구 홍제3동",
    "양천구 신월1동",
    "마포구 서교동",
    "송파구 잠실본동",
    "송파구 잠실6동",
    "양천구 신월1동",
    "마포구 망원1동",
    "서대문구 홍제3동",
    "송파구 잠실본동",
    "양천구 목3동",
    "성동구 성수1가2동",
    "서울대공원 미팅브릿지2",
    "양천구 신월1동",
    "마포구 망원1동",
    "서울대공원 발렛파킹1",
    "노원구 공릉1동",
    "도봉구 창2동",
    "도봉구 창2동",
    "강동구 천호1동",
    "광진구 중곡1동",
    "노원구 공릉1동",
    "도봉구 창3동",
    "노원구 상계2동",
    "강동구 성내1동",
    "강동구 암사3동",
    "강남구 압구정동",
    "강동구 암사1동",
    "도봉구 창2동",
    "관악구 신원동",
    "관악구 신원동",
    "도봉구 창3동",
    "강서구 가양1동",
    "강서구 방화1동",
    "강서구 등촌1동",
    "종로구 동선동",
    "강서구 공항동",
    "강서구 화곡1동",
    "도봉구 창1동",
    "구로구 고척2동",
    "금천구 독산4동",
    "구로구 고척2동",
    "종로구 이화동",
    "강서구 방화3동",
    "강동구 둔촌1동",
    "금천구 시흥2동",
    "광진구 화양동",
    "금천구 독산4동",
    "종로구 삼청동",
    "종로구 사직동",
    "강동구 암사1동",
    "종로구 가회동",
    "강동구 암사1동",
    "광진구 구의1동",
    "은평구 녹번동",
    "서울대공원 트레일",
    "관악구 낙성대동",
    "양천구 신정4동",
    "중구 신당동",
    "강북구 미아동",
    "중구 광희동",
    "강북구 수유1동",
    "중구 회현동",
    "중구 소공동",
    "용산구 이태원2동",
    "서울대공원 미팅브릿지1",
    "중구 명동",
    "중구 명동",
    "강북구 번3동",
    "서울대공원 스카이라프트",
    "광진구 광장동",
    "강북구 미아동",
    "중랑구 중화2동",
    "서울대공원 여성주차장2",
    "서울대공원 여성주차장1",
    "중랑구 묵2동",
    "강남구 신사동",
    "동작구 상도1동",
    "강남구 역삼1동",
    "강남구 역삼2동"
]


pd.DataFrame(
    {'a':distric,
     'b': admin_distric,
     'c': data
    }
)

,a,b,c
0,Gangnam-gu,Yeoksam2-dong,강남구 역삼2동
1,Dongjak-gu,Sadang2-dong,동작구 사당2동
2,Dongjak-gu,Sang-do4-dong,동작구 상도4동
3,Dongjak-gu,Sadang2-dong,동작구 사당2동
4,Dongjak-gu,Sang-do1-dong,동작구 상도1동
...,...,...,...
95,Jungnang-gu,Muk2-dong,중랑구 묵2동
96,Gangnam-gu,Sinsa-dong,강남구 신사동
97,Dongjak-gu,Sang-do1-dong,동작구 상도1동
98,Gangnam-gu,Yeoksam1-dong,강남구 역삼1동


In [30]:
distric

['Gangnam-gu']